In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
import copy

In [2]:
def show_v_table_small(v_table, env):
    for i in range(env.reward.shape[0]):        
        print("+----------"*env.reward.shape[1])
        print("|", end="")
        for j in range(env.reward.shape[1]):
            print("{0:8.2f}  |".format(v_table[i,j]),end="")
        print()
    print("+----------"*env.reward.shape[1])

# V table 그리기    
def show_v_table(v_table, env):    
    for i in range(env.reward.shape[0]):        
        print("+-----------------"*env.reward.shape[1],end="")
        print("+")
        for k in range(3):
            print("|",end="")
            for j in range(env.reward.shape[1]):
                if k==0:
                    print("                 |",end="")
                if k==1:
                        print("   {0:8.2f}      |".format(v_table[i,j]),end="")
                if k==2:
                    print("                 |",end="")
            print()
    print("+-----------------"*env.reward.shape[1],end="")
    print("+")
    
# Q table 그리기
def show_q_table(q_table,env):
    for i in range(env.reward.shape[0]):
        print("+-----------------"*env.reward.shape[1],end="")
        print("+")
        for k in range(3):
            print("|",end="")
            for j in range(env.reward.shape[1]):
                if k==0:
                    print("{0:10.2f}       |".format(q_table[i,j,0]),end="")
                if k==1:
                    print("{0:6.2f}    {1:6.2f} |".format(q_table[i,j,3],q_table[i,j,1]),end="")
                if k==2:
                    print("{0:10.2f}       |".format(q_table[i,j,2]),end="")
            print()
    print("+-----------------"*env.reward.shape[1],end="")
    print("+")
    

# 정책 policy 화살표로 그리기
def show_q_table_arrow(q_table,env):
    for i in range(env.reward.shape[0]):        
        print("+-----------------"*env.reward.shape[1],end="")
        print("+")
        for k in range(3):
            print("|",end="")
            for j in range(env.reward.shape[1]):
                if k==0:
                    if np.max(q[i,j,:]) == q[i,j,0]:
                        print("        ↑       |",end="")
                    else:
                        print("                 |",end="")
                if k==1:                    
                    if np.max(q[i,j,:]) == q[i,j,1] and np.max(q[i,j,:]) == q[i,j,3]:
                        print("      ←  →     |",end="")
                    elif np.max(q[i,j,:]) == q[i,j,1]:
                        print("          →     |",end="")
                    elif np.max(q[i,j,:]) == q[i,j,3]:
                        print("      ←         |",end="")
                    else:
                        print("                 |",end="")
                if k==2:
                    if np.max(q[i,j,:]) == q[i,j,2]:
                        print("        ↓       |",end="")
                    else:
                        print("                 |",end="")
            print()
    print("+-----------------"*env.reward.shape[1],end="")
    print("+")    
    
# 정책 policy 화살표로 그리기
def show_policy_small(policy,env):
    for i in range(env.reward.shape[0]):        
        print("+----------"*env.reward.shape[1],end="")
        print("+")
        print("|", end="")
        for j in range(env.reward.shape[1]):
            if env.reward_list1[i][j] == "road":
                if policy[i,j] == 0:
                    print("   ↑     |",end="")
                elif policy[i,j] == 1:
                    print("   →     |",end="")
                elif policy[i,j] == 2:
                    print("   ↓     |",end="")
                elif policy[i,j] == 3:
                    print("   ←     |",end="")
            else:
                print("          |",end="")
        print()
    print("+----------"*env.reward.shape[1],end="")
    print("+")
    
# 정책 policy 화살표로 그리기
def show_policy(policy,env):
    for i in range(env.reward.shape[0]):        
        print("+-----------------"*env.reward.shape[1],end="")
        print("+")
        for k in range(3):
            print("|",end="")
            for j in range(env.reward.shape[1]):
                if k==0:
                    print("                 |",end="")
                if k==1:
                    if policy[i,j] == 0:
                        print("      ↑         |",end="")
                    elif policy[i,j] == 1:
                        print("      →         |",end="")
                    elif policy[i,j] == 2:
                        print("      ↓         |",end="")
                    elif policy[i,j] == 3:
                        print("      ←         |",end="")
                if k==2:
                    print("                 |",end="")
            print()
    print("+-----------------"*env.reward.shape[1],end="")
    print("+")

In [3]:
class Agent():
    
    # 1. 행동에 따른 에이전트의 좌표 이동(위, 오른쪽, 아래, 왼쪽) 
    action = np.array([[-1,0],[0,1],[1,0],[0,-1]])
    
    # 2. 각 행동별 선택확률
    select_action_pr = np.array([0.25,0.25,0.25,0.25])
    
    # 3. 에이전트의 초기 위치 저장
    def __init__(self):
        self.pos = (0,0)
    
    # 4. 에이전트의 위치 저장
    def set_pos(self,position):
        self.pos = position
        return self.pos
    
    # 5. 에이전트의 위치 불러오기
    def get_pos(self):
        return self.pos

In [4]:
class Environment():
    
    # 1. 미로밖(절벽), 길, 목적지와 보상 설정
    cliff = -3
    road = -1
    goal = 1
    
    # 2. 목적지 좌표 설정
    goal_position = [2,2]
    
    # 3. 보상 리스트 숫자
    reward_list = [[road,road,road],
                   [road,road,road],
                   [road,road,goal]]
    
    # 4. 보상 리스트 문자
    reward_list1 = [["road","road","road"],
                    ["road","road","road"],
                    ["road","road","goal"]]
    
    # 5. 보상 리스트를 array로 설정
    def __init__(self):
        self.reward = np.asarray(self.reward_list)    

    # 6. 선택된 에이전트의 행동 결과 반환 (미로밖일 경우 이전 좌표로 다시 복귀)
    def move(self, agent, action):
        
        done = False
        
        # 6.1 행동에 따른 좌표 구하기
        new_pos = agent.pos + agent.action[action]
        
        # 6.2 현재좌표가 목적지 인지확인
        if self.reward_list1[agent.pos[0]][agent.pos[1]] == "goal":
            reward = self.goal
            observation = agent.set_pos(agent.pos)
            done = True
        # 6.3 이동 후 좌표가 미로 밖인 확인    
        elif new_pos[0] < 0 or new_pos[0] >= self.reward.shape[0] or new_pos[1] < 0 or new_pos[1] >= self.reward.shape[1]:
            reward = self.cliff
            observation = agent.set_pos(agent.pos)
            done = True
        # 6.4 이동 후 좌표가 길이라면
        else:
            observation = agent.set_pos(new_pos)
            reward = self.reward[observation[0],observation[1]]
            
        return observation, reward, done

In [5]:
#TD(0) prediction
np.random.seed(0)
env = Environment()
agent = Agent()
gamma = 0.9

V = np.zeros((3, 3))

max_episode = 10000
max_step = 100

alpha = 0.01

print("start TD(0) prediction")

for epi in tqdm(range(max_episode)):
    delta = 0
    i = 0
    j = 0
    agent.set_pos([i, j])

    for k in range(max_step):
        pos = agent.get_pos()
        action = np.random.randint(0, 4)
        observation, reward, done = env.move(agent, action)
        V[pos[0], pos[1]] += alpha * (reward + gamma * V[observation[0], observation[1]] - V[pos[0], pos[1]])
        if done:
            break
print("V(s)")
show_v_table(np.round(V, 2), env)

start TD(0) prediction


100%|██████████| 10000/10000 [00:00<00:00, 20186.26it/s]

V(s)
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|     -12.59      |     -11.01      |     -10.12      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|     -10.85      |      -8.53      |      -5.80      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|      -9.68      |      -6.06      |       3.48      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+


In [6]:
#epsilon-greedy
def e_greedy(Q_table, agent, epsilon):
    pos = agent.get_pos()
    greedy_action = np.argmax(Q_table[pos[0], pos[1], :])
    pr = np.zeros(4)
    for i in range(len(agent.action)):
        if i == greedy_action:
            pr[i] = 1- epsilon + epsilon/4
        else:
            pr[i] = epsilon/4
    return np.random.choice(4, p=pr)
#greedy
def greedy(Q_table, agent):
    pos = agent.get_pos()
    return np.argmax(Q_table[pos[0], pos[1], :])

In [7]:
#TD(0) control : SARSA
np.random.seed(0)
env = Environment()
agent = Agent()
gamma = 0.9

#Q-table
Q_table = np.random.rand(3, 3, 4)
#Q(terminal state) = 0
Q_table[2, 2, :] = 0

max_episode = 10000
max_step = 100
print("start SARSA")
alpha = 0.1
epsilon = 0.8

#every episode
for epi in tqdm(range(max_episode)):
    delta = 0
    #initialize s
    i = 0
    j = 0
    agent.set_pos([i, j])
    temp = 0

    #choose action
    action = e_greedy(Q_table, agent, epsilon)
    #every step
    for k in range(max_step):
        pos = agent.get_pos()
        #move
        observation, reward, done = env.move(agent, action)
        #choose action
        action_next = e_greedy(Q_table, agent, epsilon)
        #update Q-table
        Q_table[pos[0], pos[1], action] += alpha * (reward + gamma * Q_table[observation[0], observation[1], action_next] - Q_table[pos[0], pos[1], action])
        #update s
        agent.set_pos(observation)
        #update action
        action = action_next
        #update delta
        delta += reward
        #terminal state
        if done:
            break
#optimal policy
optimal_policy = np.zeros((3, 3))
for i in range(3):
    for j in range(3):
        optimal_policy[i, j] = np.argmax(Q_table[i, j, :])

print("SARSA : Q(s, a)")
show_q_table(np.round(Q_table, 2), env)
print("SARSA : Optimal Policy")
show_policy(optimal_policy, env)

start SARSA


100%|██████████| 10000/10000 [00:02<00:00, 3897.14it/s]

SARSA : Q(s, a)
+-----------------+-----------------+-----------------+
|     -9.20       |     -7.30       |     -5.16       |
| -9.71     -5.47 | -7.55     -4.38 | -5.91     -6.55 |
|     -5.24       |     -3.63       |      0.40       |
+-----------------+-----------------+-----------------+
|     -6.74       |     -4.63       |     -4.46       |
| -7.34     -3.18 | -5.42      3.59 | -3.82     -1.42 |
|     -3.02       |     -1.13       |      9.70       |
+-----------------+-----------------+-----------------+
|     -5.06       |     -3.86       |      9.63       |
| -6.74     -1.92 | -4.77      9.68 |  9.66      9.68 |
|     -5.29       |     -1.65       |      9.69       |
+-----------------+-----------------+-----------------+
SARSA : Optimal Policy
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|      ↓         |      ↓         |      ↓         |
|                 |                 |                 |
+-----------

In [8]:
#TD(0) control : Q-learning
np.random.seed(0)
env = Environment()
agent = Agent()
gamma = 0.9

#initialize Q-table
Q_table = np.random.rand(3, 3, 4)
#Q(terminal state) = 0
Q_table[2, 2, :] = 0

max_episode = 10000
max_step = 100
print("start Q-learning")
alpha = 0.1
epsilon = 0.8

#every episode
for epi in tqdm(range(max_episode)):
    delta = 0
    #initialize s
    i = 0
    j = 0
    agent.set_pos([i, j])

    #every step
    for k in range(max_step):
        pos = agent.get_pos()
        #choose action
        action = e_greedy(Q_table, agent, epsilon)
        #move
        observation, reward, done = env.move(agent, action)
        #next action
        next_action = greedy(Q_table, agent)
        #update Q-table
        Q_table[pos[0], pos[1], action] += alpha * (reward + gamma * Q_table[observation[0], observation[1], next_action] - Q_table[pos[0], pos[1], action])
        #update s
        agent.set_pos(observation)
        #update delta
        delta += reward
        #terminal state
        if done:
            break
#optimal policy
optimal_policy = np.zeros((3, 3))
for i in range(3):
    for j in range(3):
        optimal_policy[i, j] = np.argmax(Q_table[i, j, :])

print("Q-learning : Q(s, a)")
show_q_table(np.round(Q_table, 2), env)
print("Q-learning : Optimal Policy")
show_policy(optimal_policy, env)


start Q-learning


100%|██████████| 10000/10000 [00:02<00:00, 4436.21it/s]

Q-learning : Q(s, a)
+-----------------+-----------------+-----------------+
|      1.10       |      2.56       |      4.17       |
|  1.10      4.56 |  3.10      6.17 |  4.55      4.17 |
|      4.56       |      6.18       |      7.97       |
+-----------------+-----------------+-----------------+
|      3.10       |      4.56       |      6.17       |
|  2.56      6.18 |  4.56      7.98 |  6.18      5.97 |
|      6.18       |      7.97       |      9.97       |
+-----------------+-----------------+-----------------+
|      4.56       |      6.18       |      9.97       |
|  4.17      7.97 |  6.17      9.97 |  9.97      9.97 |
|      4.17       |      5.97       |      9.97       |
+-----------------+-----------------+-----------------+
Q-learning : Optimal Policy
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|      ↓         |      ↓         |      ↓         |
|                 |                 |                 |
+-

In [9]:
#double Q-learning
np.random.seed(0)
env = Environment()
agent = Agent()
gamma = 0.9

#initialize Q-table
Q1_table = np.random.rand(3, 3, 4)
Q2_table = np.random.rand(3, 3, 4)
#Q(terminal state) = 0
Q1_table[2, 2, :] = 0
Q2_table[2, 2, :] = 0

max_episode = 10000
max_step = 10

print("start double Q-learning")
alpha = 0.1
epsilon = 0.3

#every episode
for epi in tqdm(range(max_episode)):
    delta = 0
    #initialize s
    i = 0
    j = 0
    agent.set_pos([i, j])

    #every step
    for k in range(max_step):
        pos = agent.get_pos()
        #choose action
        action = e_greedy(Q1_table, agent, epsilon)
        #move
        observation, reward, done = env.move(agent, action)
        p = np.random.random()
        #update Q-table
        if(p<0.5):
            next_action = greedy(Q1_table, agent)
            Q1_table[pos[0], pos[1], action] += alpha * (reward + gamma * Q2_table[observation[0], observation[1], next_action] - Q1_table[pos[0], pos[1], action])
        else:
            next_action = greedy(Q2_table, agent)
            Q2_table[pos[0], pos[1], action] += alpha * (reward + gamma * Q1_table[observation[0], observation[1], next_action] - Q2_table[pos[0], pos[1], action])
        if done:
            break

optimal_policy = np.zeros((3, 3))
for i in range(3):
    for j in range(3):
        optimal_policy[i, j] = np.argmax(Q1_table[i, j, :])
print("double Q-learning : Q1(s, a)")
show_q_table(np.round(Q1_table, 2), env)
print("double Q-learning : Q2(s, a)")
show_q_table(np.round(Q2_table, 2), env)
print("double Q-learning : Q(s, a)")
show_q_table(np.round(Q1_table + Q2_table, 2), env)
print("double Q-learning : Optimal Policy")
show_policy(optimal_policy, env)

start double Q-learning


100%|██████████| 10000/10000 [00:02<00:00, 4903.67it/s]

double Q-learning : Q1(s, a)
+-----------------+-----------------+-----------------+
|      1.12       |      2.58       |      4.16       |
|  1.12      4.58 |  3.12      6.20 |  4.54      4.12 |
|      4.58       |      6.20       |      8.00       |
+-----------------+-----------------+-----------------+
|      3.11       |      4.58       |      6.20       |
|  2.53      6.11 |  4.58      8.00 |  6.20      6.00 |
|      6.20       |      8.00       |     10.00       |
+-----------------+-----------------+-----------------+
|      4.40       |      6.16       |     10.00       |
|  4.08      8.00 |  6.19     10.00 | 10.00     10.00 |
|      4.14       |      5.96       |     10.00       |
+-----------------+-----------------+-----------------+
double Q-learning : Q2(s, a)
+-----------------+-----------------+-----------------+
|      1.12       |      2.58       |      4.11       |
|  1.12      4.58 |  3.12      6.20 |  4.50      4.08 |
|      4.58       |      6.20       |      8.0

In [10]:
#actor-critic
np.random.seed(0)
env = Environment()
agent = Agent()
gamma = 0.9
#p(s, a), V(s) random
V = np.random.rand(3, 3)
policy = np.random.rand(3, 3, 4)

#policy sum = 1
for i in range(3):
    for j in range(3):
        policy[i, j, :] /= np.sum(policy[i, j, :])

max_episode = 10000
max_step = 100

print("start actor-critic")
alpha = 0.1

#every episode
for epi in tqdm(range(max_episode)):
    #initialize s
    i = 0
    j = 0
    agent.set_pos([i, j])

    #every step
    for k in range(max_step):
        #action
        pos = agent.get_pos()
        pr = np.zeros(4)
        #gibbs softmax function
        for i in range(4):
            pr[i] = np.exp(policy[pos[0], pos[1], i])/np.sum(np.exp(policy[pos[0], pos[1], :]))
        #choose action
        action = np.random.choice(4, p=pr)
        #move
        observation, reward, done = env.move(agent, action)
        #critic learning
        td_error = reward + gamma * V[observation[0], observation[1]] - V[pos[0], pos[1]]
        #V(s) = V(s) + alpha * td_error
        V[pos[0], pos[1]] += alpha * td_error
        #actor learning
        policy[pos[0], pos[1], action] += td_error * 0.01
        #minus to plus
        if np.min(policy[pos[0], pos[1], :]) < 0:
            policy[pos[0], pos[1], :] -= np.min(policy[pos[0], pos[1], :])
        for i in range(3):
            for j in range(3):
                policy[i, j, :] /= np.sum(policy[i, j, :])
        #s = final state
        if done:
            break
#optimal policy
optimal_policy = np.zeros((3, 3))
for i in range(3):
    for j in range(3):
        optimal_policy[i, j] = np.argmax(policy[i, j, :])
print("actor-critic : V(s)")
show_v_table(np.round(V, 2), env)
print("actor-critic : Policy")
show_q_table(np.round(policy, 2), env)
print("actor-critic : Optimal Policy")
show_policy(optimal_policy, env)

start actor-critic


100%|██████████| 10000/10000 [00:04<00:00, 2153.26it/s]

actor-critic : V(s)
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|      -5.81      |      -4.33      |      -2.75      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|      -4.22      |      -2.08      |       2.15      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|      -2.06      |       1.76      |      10.00      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
actor-critic : Policy
+-----------------+-----------------+-----------------+
|      0.02       |      0.00       |      0.00       |
|  0.00      0.49 |  0.05      0.36 |  0.13      0.08 |
|      0.48       |      0.58       |      0.79       |
+-----

In [11]:
#TD(0) function approximation
np.random.seed(1)
env = Environment()
agent = Agent()
gamma = 0.9

#initialize
#w[0] + w[1]*x1 + w[2]*x2
w = np.random.rand(3)
w -=0.5

v_table = np.zeros((3, 3))
for i in range(3):
    for j in range(3):
        v_table[i, j] = np.dot(w, [1, i, j])
max_episode = 10000
max_step = 100
alpha = 0.01
epsilon = 0.3
print("start TD(0) function approximation")
#every episode
for epi in tqdm(range(max_episode)):
    delta = 0
    #initialize s
    i = 0
    j = 0
    agent.set_pos([i, j])
    temp = 0

    #every step
    for k in range(max_step):
        pos = agent.get_pos()
        #choose action randomly
        action = np.random.choice(4)
        #move
        observation, reward, done = env.move(agent, action)
        now_v = 0
        next_v = 0

        #get now_v and next_v with V(s|w)
        now_v = w[0] + np.dot(w[1:], pos)
        next_v = w[0] + np.dot(w[1:], observation)

        #w = w+alpha*(reward+gamma*next_v-now_v)*x
        w[0] = w[0] + alpha * (reward + gamma * next_v - now_v)
        w[1] = w[1] + alpha * (reward + gamma * next_v - now_v) * pos[0]
        w[2] = w[2] + alpha * (reward + gamma * next_v - now_v) * pos[1]

        #s = final state
        if done:
            break
for i in range(3):
    for j in range(3):
        v_table[i, j] = np.dot(w, [1, i, j])
print("TD(0) function approximation : V(s)")
show_v_table(np.round(v_table, 2), env)

start TD(0) function approximation


100%|██████████| 10000/10000 [00:00<00:00, 11933.61it/s]

TD(0) function approximation : V(s)
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|     -13.92      |     -12.68      |     -11.44      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|     -12.26      |     -11.02      |      -9.78      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|     -10.60      |      -9.36      |      -8.12      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+


In [14]:
#Q-learning function approximation
np.random.seed(0)
#initialize
env = Environment()
agent = Agent()
gamma = 0.9
#q(s, a|w) <- differentiable function
#w <- random value
w = np.random.rand(len(agent.action), env.reward.shape[0])
w -= 0.5
FA_Q_table = np.zeros((3, 3, 4))
#function save to table
for i in range(3):
    for j in range(3):
        for k in range(4):
            FA_Q_table[i, j, k] = np.dot(w[k, :], [1, i, j])
#get optimal policy
optimal_policy = np.zeros((3, 3))
for i in range(3):
    for j in range(3):
        optimal_policy[i, j] = np.argmax(FA_Q_table[i, j, :])

print("Before : function approximation Q-learning : Q(s, a|w)")
show_q_table(np.round(FA_Q_table, 2), env)
print()
print("Before : function approximation Q-learning : Policy")
show_policy(optimal_policy, env)
print()
print("initial w")
print("w = {}".format(np.round(w, 2)))
print()

max_episode = 100000
max_step = 100

print("start function approximation Q-learning")
alpha = 0.01

#every episode
for epi in tqdm(range(max_episode)):
    #initial state
    i = 0
    j = 0
    agent.set_pos([i, j])
    #every step
    for k in range(max_step):
        pos = agent.get_pos()
        #choose action with gibbs softmax function
        action = np.zeros(4)
        for act in range(4):
            action[act] = np.dot(w[act, :], [1, pos[0], pos[1]])
        pr = np.zeros(4)
        for i in range(4):
            pr[i] = np.exp(action[i])/np.sum(np.exp(action))
        
        action = np.random.choice(4, p=pr)
        #move
        observation, reward, done = env.move(agent, action)
        #choose target policy action : greedy
        next_act = np.zeros(4)
        for act in range(4):
            next_act[act] = np.dot(w[act, 1:], observation) + w[act, 0]
        best_action = np.argmax(next_act)

        now_q = np.dot(w[action, 1:], pos) + w[action, 0]
        next_q = np.dot(w[best_action, 1:], observation) + w[best_action, 0]
        #w = w+alpha*(reward+gamma*next_q-now_q)*x
        w[action, 0] = w[action, 0] + alpha * (reward + gamma * next_q - now_q)
        w[action, 1] = w[action, 1] + alpha * (reward + gamma * next_q - now_q) * pos[0]
        w[action, 2] = w[action, 2] + alpha * (reward + gamma * next_q - now_q) * pos[1]
        #s = final state
        if done:
            break
#FA_Q = np.zeros((4, 3, 3))
#function save to table
for i in range(3):
    for j in range(3):
        for k in range(4):
            FA_Q_table[i, j, k] = np.dot(w[k, :], [1, i, j])

#optimal policy
for i in range(3):
    for j in range(3):
        optimal_policy[i, j] = np.argmax(FA_Q_table[i, j, :])
print("After : function approximation Q-learning : Q(s, a|w)")
show_q_table(np.round(FA_Q_table, 2), env)
print()
print("After : function approximation Q-learning : Policy")
show_policy(optimal_policy, env)
print()
print("final w")
print("w = {}".format(np.round(w, 2)))
print()

Before : function approximation Q-learning : Q(s, a|w)
+-----------------+-----------------+-----------------+
|      0.05       |      0.15       |      0.25       |
| -0.12      0.04 | -0.09      0.19 | -0.06      0.34 |
|     -0.06       |      0.40       |      0.86       |
+-----------------+-----------------+-----------------+
|      0.26       |      0.37       |      0.47       |
|  0.18     -0.03 |  0.20      0.11 |  0.23      0.26 |
|      0.33       |      0.79       |      1.26       |
+-----------------+-----------------+-----------------+
|      0.48       |      0.58       |      0.68       |
|  0.47     -0.11 |  0.50      0.04 |  0.52      0.18 |
|      0.72       |      1.18       |      1.65       |
+-----------------+-----------------+-----------------+

Before : function approximation Q-learning : Policy
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|      ↑         |      ↓         |      ↓         |

100%|██████████| 100000/100000 [00:58<00:00, 1712.66it/s]

After : function approximation Q-learning : Q(s, a|w)
+-----------------+-----------------+-----------------+
|     -9.44       |     -9.07       |     -8.69       |
| -7.18      4.76 | -3.16      5.53 |  0.86      6.31 |
|      4.68       |      6.72       |      8.77       |
+-----------------+-----------------+-----------------+
|     -8.07       |     -7.69       |     -7.32       |
| -2.28      6.69 |  1.74      7.47 |  5.75      8.25 |
|      5.42       |      7.46       |      9.50       |
+-----------------+-----------------+-----------------+
|     -6.69       |     -6.32       |     -5.95       |
|  2.62      8.63 |  6.63      9.41 | 10.65     10.18 |
|      6.15       |      8.20       |     10.24       |
+-----------------+-----------------+-----------------+

After : function approximation Q-learning : Policy
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|      →         |      ↓         |      ↓         |
|